In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import tensorflow as tf
from keras.layers import Input, Dense
from keras.models import Sequential, Model
from tensorflow.keras import datasets
from tensorflow.keras.layers import Dense

In [ ]:
hack_img = np.ones((28,28))*0

In [ ]:
# 載入原本的模組
load_model = tf.keras.models.load_model('.\\mnist_basic_model.h5')
load_model.trainable = False
load_model.summary()

In [ ]:
# 想辦法從模型中萃取出預設為 9 的輸入資料
# 依照 AutoEncoder 想法, 建立一個輸入長度為10的模型, 在串接一個 784 的 Dense
attack_vector = Input(shape=(10,))
output_vector = Dense(28 * 28, activation='relu', input_dim=10)(attack_vector)
attack_model = Model(inputs=attack_vector, outputs=output_vector)
attack_model.summary()

attack_model.save('attack_model.h5',save_format='h5')

In [ ]:
# 把兩個模型接再一起
target_output = load_model(output_vector)
combined_model = Model(inputs=attack_vector, outputs=target_output)
combined_model.compile(loss='binary_crossentropy', optimizer=tf.optimizers.Adam())
combined_model.summary()

combined_model.save('combined_model.h5',save_format='h5')

In [ ]:
# 開始用 9 的資料訓練這個模型
batch_size = 128
total_epochs = 1000

final_target = np.zeros((batch_size, 10))
for i in range(batch_size):
    final_target[i][9] = 0.9 + np.random.random() * 0.1


combined_model.fit(final_target,final_target, epochs=total_epochs, batch_size=batch_size)

In [ ]:
hack_data = attack_model.predict(final_target[0:2])
hack_img = np.reshape(hack_data[0],(28,28))

#????

In [ ]:
hack_img = hack_img.astype(np.uint8)
im = Image.fromarray(hack_img)
im.save("hack.bmp")

test_data = np.asarray(hack_img)
plt.imshow(test_data,cmap='gray')